# logistic回归
使用logistic回归对结构化数据进行简单的分类。

## 1. 模型介绍
Logistic Regression 虽然被称为回归，但其实际上是分类模型，并常用于二分类。Logistic Regression 因其简单、可并行化、可解释强深受工业界喜爱。
Logistic 回归的本质是：假设数据服从这个分布，然后使用极大似然估计做参数的估计。Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0，所以这里会用到它。

**Logistic 回归的本质是：假设数据服从这个分布，然后使用极大似然估计做参数的估计。**

logistic回归可以用于通过相似度数据来得到图片是否相似，也就是对于多个数据进行计算得到相似与不相似两个结果，由于本人对于机器学习造诣不深，所以不会涉及回归模型的原理讲解。

[了解更多](https://zhuanlan.zhihu.com/p/74874291#:~:text=%E4%B9%8B%E5%89%8D%E8%AF%B4%E5%88%B0Logistic%20%E5%9B%9E%E5%BD%92%E4%B8%BB%E8%A6%81%E7%94%A8%E4%BA%8E%E5%88%86%E7%B1%BB%E9%97%AE%E9%A2%98%EF%BC%8C%E6%88%91%E4%BB%AC%E4%BB%A5%E4%BA%8C%E5%88%86%E7%B1%BB%E4%B8%BA%E4%BE%8B%EF%BC%8C%E5%AF%B9%E4%BA%8E%E6%89%80%E7%BB%99%E6%95%B0%E6%8D%AE%E9%9B%86%E5%81%87%E8%AE%BE%E5%AD%98%E5%9C%A8%E8%BF%99%E6%A0%B7%E7%9A%84%E4%B8%80%E6%9D%A1%E7%9B%B4%E7%BA%BF%E5%8F%AF%E4%BB%A5%E5%B0%86%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90%E7%BA%BF%E6%80%A7%E5%8F%AF%E5%88%86%E3%80%82%20%E9%82%A3%E4%B9%88%E5%8F%AF%E4%BB%A5%E5%88%A4%E6%96%AD%E5%AE%83%E7%9A%84%E7%B1%BB%E5%88%AB%E4%B8%BA1%EF%BC%8C%E8%BF%99%E4%B8%AA%E8%BF%87%E7%A8%8B%E5%85%B6%E5%AE%9E%E6%98%AF%E6%84%9F%E7%9F%A5%E6%9C%BA%E3%80%82,Logistic%20%E5%9B%9E%E5%BD%92%E8%BF%98%E9%9C%80%E8%A6%81%E5%8A%A0%E4%B8%80%E5%B1%82%EF%BC%8C%E5%AE%83%E8%A6%81%E6%89%BE%E5%88%B0%E5%88%86%E7%B1%BB%E6%A6%82%E7%8E%87P%28Y%3D1%29%20%E4%B8%8E%E8%BE%93%E5%85%A5%E5%90%91%E9%87%8Fx%20%E7%9A%84%E7%9B%B4%E6%8E%A5%E5%85%B3%E7%B3%BB%EF%BC%8C%E7%84%B6%E5%90%8E%E9%80%9A%E8%BF%87%E6%AF%94%E8%BE%83%E6%A6%82%E7%8E%87%E5%80%BC%E6%9D%A5%E5%88%A4%E6%96%AD%E7%B1%BB%E5%88%AB%E3%80%82)

## 2.UCI German Credit  数据集

UCI German Credit是UCI的德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据是根据个人的银行贷款信息和申请客户贷款逾期发生情况来预测贷款违约倾向的数据集，数据集包含24个维度的，1000条数据，

在这里我们直接使用处理好的数值化的数据，作为展示。

In [50]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.10.2'

## 3.代码实战
我们这里使用的 german.data-numeric是numpy处理好数值化数据，我们直接使用numpy的load方法读取即可

同样的，你也可以从这里下载
[资源](https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/)

In [51]:
data=np.loadtxt("./resource/german.data-numeric")
print(data)

[[ 1.  6.  4. ...  0.  1.  1.]
 [ 2. 48.  2. ...  0.  1.  2.]
 [ 4. 12.  4. ...  1.  0.  1.]
 ...
 [ 4. 12.  2. ...  0.  1.  1.]
 [ 1. 45.  2. ...  0.  1.  2.]
 [ 2. 45.  4. ...  0.  1.  1.]]


In [52]:
h,w=data.shape
print(h,w)

1000 25


读取完数据后，要对其进行归一化处理。

In [53]:
for j in range(w-1):
    meanVal=np.mean(data[:,j])  #获取每一列的平均值
    stdVal=np.std(data[:,j])    #获取每一列的标准差
    data[:,j]=(data[:,j]-meanVal)/stdVal   #z标准化

打乱数据

In [54]:
np.random.shuffle(data)

区分训练集和测试集，由于这里没有验证集，所以我们直接使用测试集的准确度作为评判好坏的标准

区分规则：900条用于训练，100条作为测试

german.data-numeric的格式为，前24列为24个维度，最后一个为要打的标签（0，1），所以我们将数据和标签一起区分出来

In [55]:
train_data=data[:900,:w-1]   #列取900，横取24
train_lab=data[:900,w-1]-1   
test_data=data[900:,:w-1]
test_lab=data[900:,w-1]-1

下面我们定义模型，模型很简单

In [56]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(24,2) # 由于24个维度已经固定了，所以这里写24
    def forward(self,x):
        out=self.fc(x)
        out=torch.sigmoid(out)
        return out

测试集上的准确率

In [57]:
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

下面就是对一些设置

In [58]:
net=LR() 
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000 # 训练1000次

下面开始训练了

In [59]:
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))

Epoch:100,Loss:0.6480,Accuracy：0.68
Epoch:200,Loss:0.6180,Accuracy：0.73
Epoch:300,Loss:0.6000,Accuracy：0.76
Epoch:400,Loss:0.5877,Accuracy：0.76
Epoch:500,Loss:0.5785,Accuracy：0.77
Epoch:600,Loss:0.5713,Accuracy：0.77
Epoch:700,Loss:0.5653,Accuracy：0.77
Epoch:800,Loss:0.5603,Accuracy：0.77
Epoch:900,Loss:0.5560,Accuracy：0.77
Epoch:1000,Loss:0.5523,Accuracy：0.79


训练完成了，我们的准确度达到了79%（使用pycharm，得到的是80%，和打乱数据有关，相差不大）